In [2]:
import requests
import pandas as pd

# create request header
headers = {'User-Agent': "email@address.com"}




### Get company tickers

In [3]:
# get all companies data
companyTickers = requests.get(
    "https://www.sec.gov/files/company_tickers.json",
    headers=headers
    )
company_tickers = pd.DataFrame(companyTickers.json()).T.set_index("cik_str")
company_tickers


,ticker,title
cik_str,,
789019,MSFT,MICROSOFT CORP
320193,AAPL,Apple Inc.
1045810,NVDA,NVIDIA CORP
1018724,AMZN,AMAZON COM INC
1652044,GOOGL,Alphabet Inc.
...,...,...
1901336,ALCYW,Alchemy Investments Acquisition Corp 1
1901336,ALCYU,Alchemy Investments Acquisition Corp 1
1977303,ARLTF,Arcadium Lithium plc


In [220]:
ticker = "AAPL"

cik_id = str(company_tickers[company_tickers["ticker"] == ticker].index[0])

cik_str=str(cik_id).zfill(10) # we use zfill since the company number must be 10 digits long

companyFacts = requests.get(
    f'https://data.sec.gov/api/xbrl/companyfacts/CIK{cik_str}.json',
    headers=headers
    )

dfs = []
for k in companyFacts.json()["facts"]["us-gaap"].keys():
    tmp = companyFacts.json()["facts"]["us-gaap"][k]["units"]
    if len(tmp.keys())>1:
        print(k,tmp.keys())
    df = pd.DataFrame(tmp[list(tmp.keys())[0]]).set_index("filed")
    df.index =  pd.to_datetime(df.index)
    df["fact"]=k   
    dfs.append(df)

company_facts = pd.concat(dfs)
company_facts.index = pd.to_datetime(company_facts.index)   



FiniteLivedIntangibleAssetsUsefulLifeMaximum dict_keys(['pure', 'Year'])
FiniteLivedIntangibleAssetsUsefulLifeMinimum dict_keys(['pure', 'Year'])
ShareBasedCompensationArrangementByShareBasedPaymentAwardOptionsExercisableWeightedAverageRemainingContractualTerm dict_keys(['pure', 'Year'])


In [211]:
q_statements = pd.read_csv("lib/stocks/quarterly_statements.csv",index_col=0,thousands=',')
q_statements = q_statements[q_statements["ticker"] == ticker]
q_statements.head()

/var/folders/t1/zq_gddfn2h9_xnb64_r95zpc0000gn/T/ipykernel_69257/567704258.py:1: DtypeWarning: Columns (10,16,28,35,36,37,38,39,41,42,43,45,46,47,48,50,51,52,53,56,62,72,111,112,120,121,122,123,124) have mixed types. Specify dtype option on import or set low_memory=False.
  q_statements = pd.read_csv("lib/stocks/quarterly_statements.csv",index_col=0,thousands=',')


,Cash & Equivalents,Short-Term Investments,Cash & Cash Equivalents,Cash Growth,Receivables,Inventory,Other Current Assets,Total Current Assets,"Property, Plant & Equipment",Long-Term Investments,...,Total Liabilities and Equity,Common Stock Issued,Share Repurchases,Interest Income,Interest Expense,Asset Turnover,Return on Equity (ROE),Return on Assets (ROA),Earnings Yield,FCF Yield
Quarter Ended,,,,,,,,,,,,,,,,,,,,,
2023-12-30,"40,760","32,340",172575.0,4.31%,"50,102","6,511","13,979","143,692","43,666","99,475",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-09-30,"29,965","31,590",162099.0,-4.15%,"60,985","6,331","14,695","143,566","43,715","100,544",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-07-01,"28,408","34,074",166543.0,-7.12%,"39,186","7,351","13,640","122,659","43,550","104,061",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2023-04-01,"24,687","31,185",166333.0,-13.70%,"35,899","7,482","13,660","112,913","43,398","110,461",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2022-12-31,"20,535","30,820",165450.0,-18.34%,"54,180","6,820","16,422","128,777","42,951","114,095",...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [469]:
company_facts

,end,val,accn,fy,fp,form,frame,fact,start
filed,,,,,,,,,
2009-07-22,2008-09-27,5.520000e+09,0001193125-09-153165,2009,Q3,10-Q,CY2008Q3I,AccountsPayable,NaN
2009-07-22,2009-06-27,4.854000e+09,0001193125-09-153165,2009,Q3,10-Q,CY2009Q2I,AccountsPayable,NaN
2009-10-27,2008-09-27,5.520000e+09,0001193125-09-214859,2009,FY,10-K,NaN,AccountsPayableCurrent,NaN
2010-01-25,2008-09-27,5.520000e+09,0001193125-10-012091,2009,FY,10-K/A,CY2008Q3I,AccountsPayableCurrent,NaN
2009-10-27,2009-09-26,5.601000e+09,0001193125-09-214859,2009,FY,10-K,NaN,AccountsPayableCurrent,NaN
...,...,...,...,...,...,...,...,...,...
2024-02-02,2022-12-31,6.299500e+07,0000320193-24-000006,2024,Q1,10-Q,CY2022Q4,IncrementalCommonSharesAttributableToShareBase...,2022-09-25
2023-11-03,2023-09-30,6.831600e+07,0000320193-23-000106,2023,FY,10-K,CY2023,IncrementalCommonSharesAttributableToShareBase...,2022-09-25
2024-02-02,2023-12-30,6.687800e+07,0000320193-24-000006,2024,Q1,10-Q,CY2023Q4,IncrementalCommonSharesAttributableToShareBase...,2023-10-01


In [522]:
company_facts_q4 = company_facts[company_facts.end=="2023-12-30"]

pd.set_option('display.max_rows', None)

display(company_facts_q4[["val","fact"]].set_index("fact").astype(str))

,val
fact,
AccountsPayableCurrent,58146000000.0
AccountsReceivableNetCurrent,23194000000.0
AccumulatedDepreciationDepletionAndAmortizationPropertyPlantAndEquipment,72510000000.0
AccumulatedOtherComprehensiveIncomeLossNetOfTax,-9378000000.0
AllocatedShareBasedCompensationExpense,2997000000.0
Assets,353514000000.0
AssetsCurrent,143692000000.0
AssetsNoncurrent,209822000000.0
AvailableForSaleSecuritiesDebtMaturitiesRollingAfterYearTenFairValue,17113000000.0


In [514]:
def find_indexes(series, target_sum):
    seen = {}
    for i, num in enumerate(series):
        complement = target_sum - num
        if complement in seen:
            return [seen[complement], i]
        seen[num] = i
    return []
    
find_indexes(company_facts_q4[["val","fact"]].set_index("fact").astype(float).val,26380000000)

[]

In [344]:
import json
def read_list_from_file(file_path):
    with open(file_path, 'r') as f:
        data = f.read()
    return json.loads(data)

features = read_list_from_file("t.txt")

In [536]:
company_facts.index = pd.to_datetime(company_facts.index)   


In [219]:
# company_facts_q4.loc[company_facts_q4.fact.str.contains("Invest")]

company_facts_q4.loc[(company_facts_q4.val <= 32350000000) | company_facts_q4.val >= 32320000000]

,end,val,accn,fy,fp,form,frame,fact,start
filed,,,,,,,,,


In [215]:
company_facts[company_facts.fact.str.contains("Investments")].sort_index(ascending=False)

,end,val,accn,fy,fp,form,frame,fact,start
filed,,,,,,,,,
2022-10-28,2022-09-24,1.270000e+10,0000320193-22-000108,2022,FY,10-K,CY2022Q3I,RestrictedInvestments,NaN
2022-10-28,2021-09-25,1.790000e+10,0000320193-22-000108,2022,FY,10-K,CY2021Q3I,RestrictedInvestments,NaN
2022-07-29,2022-06-25,1.410000e+10,0000320193-22-000070,2022,Q3,10-Q,CY2022Q2I,RestrictedInvestments,NaN
2022-07-29,2021-09-25,1.790000e+10,0000320193-22-000070,2022,Q3,10-Q,NaN,RestrictedInvestments,NaN
2022-04-29,2022-03-26,1.540000e+10,0000320193-22-000059,2022,Q2,10-Q,CY2022Q1I,RestrictedInvestments,NaN
...,...,...,...,...,...,...,...,...,...
2009-07-22,2009-06-27,6.100000e+07,0001193125-09-153165,2009,Q3,10-Q,NaN,PaymentsToAcquireOtherInvestments,2008-09-28
2009-07-22,2009-06-27,5.746000e+09,0001193125-09-153165,2009,Q3,10-Q,NaN,IncomeLossFromContinuingOperationsBeforeIncome...,2008-09-28
2009-07-22,2008-06-28,3.100000e+07,0001193125-09-153165,2009,Q3,10-Q,NaN,PaymentsToAcquireOtherInvestments,2007-09-30


In [548]:
def prepre_sec_dataframe(dataframe,fact):
    """ Prepares a sec dataframe for using in the transformation functions"""
    df = dataframe[dataframe.fact.str.match(fact)]
    df = df.sort_index(ascending=False)
    df= df.groupby("frame").last()
    df = df.reset_index().set_index('end').sort_index(ascending=False)
    return df

In [347]:
prepre_sec_dataframe(company_facts,"CashAndCashEquivalentsAtCarryingValue").val

end
2023-12-30    4.076000e+10
2023-09-30    2.996500e+10
2023-07-01    2.840800e+10
2023-04-01    2.468700e+10
2022-12-31    2.053500e+10
                  ...     
2008-12-27    7.236000e+09
2008-09-27    1.187500e+10
2008-06-28    9.373000e+09
2007-09-29    9.352000e+09
2006-09-30    6.392000e+09
Name: val, Length: 65, dtype: float64

In [343]:
features

['Cash & Equivalents',
 'Cash Growth',
 'Receivables',
 'Inventory',
 'Other Current Assets',
 'Total Current Assets',
 'Property, Plant & Equipment',
 'Goodwill and Intangibles',
 'Other Long-Term Assets',
 'Total Long-Term Assets',
 'Total Assets',
 'Accounts Payable',
 'Current Debt',
 'Other Current Liabilities',
 'Total Current Liabilities',
 'Long-Term Debt',
 'Other Long-Term Liabilities',
 'Total Long-Term Liabilities',
 'Total Liabilities',
 'Total Debt',
 'Debt Growth',
 'Retained Earnings',
 'Comprehensive Income',
 'Shareholders Equity',
 'Net Cash / Debt',
 'Net Cash Per Share',
 'Working Capital',
 'Book Value Per Share',
 'Net Income',
 'Depreciation & Amortization',
 'Share-Based Compensation',
 'Other Operating Activities',
 'Operating Cash Flow',
 'Operating Cash Flow Growth',
 'Capital Expenditures',
 'Acquisitions',
 'Other Investing Activities',
 'Investing Cash Flow',
 'Dividends Paid',
 'Share Issuance / Repurchase',
 'Debt Issued / Paid',
 'Other Financing Activ

In [497]:
{"Cash & Equivalents":prepre_sec_dataframe(company_facts,"CashAndCashEquivalentsAtCarryingValue").val,
 "Short-Term Investments":prepre_sec_dataframe(company_facts,"MarketableSecuritiesCurrent").val,
 "Cash & Cash Equivalents" :cash_and_cash_equivalents(company_facts),
 "Receivables":receivables(company_facts),
  "Inventory":inventory(company_facts),
 'Other Current Assets':prepre_sec_dataframe(company_facts,"OtherAssetsCurrent").val,
 'Total Current Assets':prepre_sec_dataframe(company_facts,"AssetsCurrent").val,
 "Property, Plant & Equipment":prepre_sec_dataframe(company_facts,"PropertyPlantAndEquipmentNet").val,
 "Goodwill and Intangibles":prepre_sec_dataframe(company_facts,"Goodwill").val,
'Other Long-Term Assets':prepre_sec_dataframe(company_facts,"OtherAssetsNoncurrent").val,
 'Total Long-Term Assets':prepre_sec_dataframe(company_facts,"AssetsNoncurrent").val,
 'Total Assets':prepre_sec_dataframe(company_facts,"Assets").val,
 'Accounts Payable':prepre_sec_dataframe(company_facts,"AccountsPayableCurrent").val,
 'Current Debt': current_debt(company_facts),
 'Other Current Liabilities':prepre_sec_dataframe(company_facts,"OtherLiabilitiesCurrent").val,
 'Total Current Liabilities':prepre_sec_dataframe(company_facts,"LiabilitiesCurrent").val,
 'Long-Term Debt':prepre_sec_dataframe(company_facts,"LongTermDebtNoncurrent").val,
 'Other Long-Term Liabilities':prepre_sec_dataframe(company_facts,"OtherLiabilitiesNoncurrent").val,
 'Total Long-Term Liabilities':prepre_sec_dataframe(company_facts,"LiabilitiesNoncurrent").val,
 'Total Liabilities':prepre_sec_dataframe(company_facts,"Liabilities").val,
'Total Debt': total_debt(company_facts),
#  'Debt Growth',
 'Retained Earnings':prepre_sec_dataframe(company_facts,"RetainedEarningsAccumulatedDeficit").val,
 'Comprehensive Income':prepre_sec_dataframe(company_facts,"ComprehensiveIncomeNetOfTax").val,
 'Shareholders Equity': shareholders_equity(company_facts),
 'Net Cash / Debt' : cash_and_cash_equivalents(company_facts) - total_debt(company_facts),
 'Net Cash Per Share':  (cash_and_cash_equivalents(company_facts) - total_debt(company_facts)) / prepre_sec_dataframe(company_facts,"CommonStockSharesOutstanding").val, # slightly differs as the value in stockanalysis uses the shares from 10-K
 'Working Capital': prepre_sec_dataframe(company_facts,"AssetsCurrent").val - prepre_sec_dataframe(company_facts,"LiabilitiesCurrent").val,
 'Book Value Per Share':  prepre_sec_dataframe(company_facts,"StockholdersEquity").val / prepre_sec_dataframe(company_facts,"CommonStockSharesOutstanding").val,
 'Net Income': prepre_sec_dataframe(company_facts,"NetIncomeLoss").val,
 'Depreciation & Amortization': prepre_sec_dataframe(company_facts,"DepreciationDepletionAndAmortization").val,
 'Share-Based Compensation': prepre_sec_dataframe(company_facts,"ShareBasedCompensation").val,
#  'Other Operating Activities',
#  'Operating Cash Flow': prepre_sec_dataframe(company_facts,"NetCashProvidedByUsedInOperatingActivities").val,
#  'Operating Cash Flow Growth',
 'Capital Expenditures': - prepre_sec_dataframe(company_facts,"PaymentsToAcquirePropertyPlantAndEquipment").val,
'Other Investing Activities': -prepre_sec_dataframe(company_facts,"PaymentsForProceedsFromOtherInvestingActivities").val, # has to be negative values
 'Investing Cash Flow': prepre_sec_dataframe(company_facts,"NetCashProvidedByUsedInInvestingActivities").val,
 'Dividends Paid': - prepre_sec_dataframe(company_facts,"PaymentsOfDividends").val, #  negative values
 'Share Issuance / Repurchase': - prepre_sec_dataframe(company_facts,"PaymentsForRepurchaseOfCommonStock").val, #negative values
 'Debt Issued / Paid': - prepre_sec_dataframe(company_facts,"PaymentsOfDebtIssuanceCosts").val, # negative values
#  'Other Financing Activities': 
 'Financing Cash Flow': prepre_sec_dataframe(company_facts,"NetCashProvidedByUsedInFinancingActivities").val,
 'Net Cash Flow':prepre_sec_dataframe(company_facts,"CashCashEquivalentsRestrictedCashAndRestrictedCashEquivalentsPeriodIncreaseDecreaseIncludingExchangeRateEffect").val,
 'Free Cash Flow': free_cash_flow(company_facts),
#  'Free Cash Flow Growth',
 'Free Cash Flow Margin': (free_cash_flow(company_facts)/ revenue(company_facts))*100,
 'Free Cash Flow Per Share': free_cash_flow(company_facts) / prepre_sec_dataframe(company_facts,"CommonStockSharesOutstanding").val,
 'Cost of Revenue': prepre_sec_dataframe(company_facts,"CostOfGoodsAndServicesSold").val,
 'Gross Profit': prepre_sec_dataframe(company_facts,"GrossProfit").val,
 'Selling, General & Admin': prepre_sec_dataframe(company_facts,"SellingGeneralAndAdministrativeExpense").val,
 'Operating Expenses': prepre_sec_dataframe(company_facts,"OperatingExpenses").val,
 'Operating Income': prepre_sec_dataframe(company_facts,"OperatingIncomeLoss").val,
#  'Interest Expense / Income',
 'Other Expense / Income': prepre_sec_dataframe(company_facts,"OtherNonoperatingIncomeExpense").val,
 'Pretax Income': prepre_sec_dataframe(company_facts,"NetIncomeLoss").val,
 'Income Tax': prepre_sec_dataframe(company_facts,"IncomeTaxExpenseBenefit").val,
 "Net Income": prepre_sec_dataframe(company_facts,"NetIncomeLoss").val,
 "Revenue": revenue(company_facts),
#  'Net Income Growth',
'Shares Outstanding (Basic)': prepre_sec_dataframe(company_facts,"WeightedAverageNumberOfSharesOutstandingBasic").val,
 'Shares Outstanding (Diluted)': prepre_sec_dataframe(company_facts,"WeightedAverageNumberOfDilutedSharesOutstanding").val,
#  'Shares Change',
 'EPS (Basic)': prepre_sec_dataframe(company_facts,"NetIncomeLoss").val / prepre_sec_dataframe(company_facts,"WeightedAverageNumberOfSharesOutstandingBasic").val,
 'EPS (Diluted)': prepre_sec_dataframe(company_facts,"NetIncomeLoss").val / prepre_sec_dataframe(company_facts,"WeightedAverageNumberOfDilutedSharesOutstanding").val,
#  'EPS Growth',
 'Dividend Per Share',
 'Gross Margin': (prepre_sec_dataframe(company_facts,"GrossProfit").val / revenue(company_facts))*100,
 'Operating Margin': (prepre_sec_dataframe(company_facts,"OperatingIncomeLoss").val / revenue(company_facts))*100,
 'Profit Margin':  (prepre_sec_dataframe(company_facts,"NetIncomeLoss").val / revenue(company_facts))*100,
 "Free Cash Flow Margin": (free_cash_flow(company_facts) / revenue(company_facts))*100,
 'Effective Tax Rate':  (prepre_sec_dataframe(company_facts,"IncomeTaxExpenseBenefit").val /  prepre_sec_dataframe(company_facts,"NetIncomeLoss").val) * 100, #Effective Tax Rate = (Income Tax / Pretax Income) * 100%
#  'EBITDA',
#  'EBITDA Margin',
#  'EBIT',
#  'EBIT Margin',
#  'Market Capitalization',
#  'Market Cap Growth',
#  'Enterprise Value',
#  }

SyntaxError: invalid syntax. Perhaps you forgot a comma? (3123981904.py, line 47)

In [529]:
"15,576,641,000".replace(",","")


'15576641000'

In [85]:
quarterly = pd.read_csv("lib/stocks/quarterly_statements.csv",index_col=0)
quarterly = quarterly[quarterly["ticker"] == ticker]
quarterly['Cash & Equivalents'][:5]

/var/folders/t1/zq_gddfn2h9_xnb64_r95zpc0000gn/T/ipykernel_69257/572592244.py:1: DtypeWarning: Columns (10,16,37,43,46,51,56,62,72,111,112,113,114,115,116,117,118,119,120,121,122,123,124) have mixed types. Specify dtype option on import or set low_memory=False.
  quarterly = pd.read_csv("lib/stocks/quarterly_statements.csv",index_col=0)


Quarter Ended
2023-12-30    40,760
2023-09-30    29,965
2023-07-01    28,408
2023-04-01    24,687
2022-12-31    20,535
Name: Cash & Equivalents, dtype: object

In [506]:
(free_cash_flow(company_facts))

end
2007-09-29             NaN
2008-09-27             NaN
2008-12-27             NaN
2009-09-26             NaN
2009-12-26             NaN
2010-09-25             NaN
2010-12-25             NaN
2011-09-24             NaN
2011-12-31             NaN
2012-09-29             NaN
2012-12-29             NaN
2013-09-28    4.550100e+10
2013-12-28    2.068500e+10
2014-09-27    5.014200e+10
2014-12-27    3.050500e+10
2015-09-26    7.001900e+10
2015-12-26    2.385100e+10
2016-09-24    5.309000e+10
2016-12-31    2.372200e+10
2017-09-30    5.177400e+10
2017-12-30    2.548300e+10
2018-09-29    6.412100e+10
2018-12-29    2.333500e+10
2019-09-28    5.889600e+10
2019-12-28    2.840900e+10
2020-09-26    7.336500e+10
2020-12-26    3.526300e+10
2021-09-25    9.295300e+10
2021-12-25    4.416300e+10
2022-09-24    1.114430e+11
2022-12-31    3.021800e+10
2023-09-30    9.958400e+10
2023-12-30    3.750300e+10
Name: val, dtype: float64

In [546]:
prepre_sec_dataframe(company_facts,"NetCashProvidedByUsedInOperatingActivities").val[:10]

end
2007-09-29    5.470000e+09
2008-09-27    9.596000e+09
2008-12-27    3.938000e+09
2009-09-26    1.015900e+10
2009-12-26    5.781000e+09
2010-09-25    1.859500e+10
2010-12-25    9.773000e+09
2011-09-24    3.752900e+10
2011-12-31    1.755400e+10
2012-09-29    5.085600e+10
Name: val, dtype: float64

In [499]:
def net_cash_debt(df,total_cash,total_debt):
    return df[total_cash] - df[total_debt]

def cash_and_cash_equivalents(df):
    short_term = prepre_sec_dataframe(df,"MarketableSecuritiesCurrent").val.astype(float) #ShortTermInvestments
    long_term= prepre_sec_dataframe(df,"MarketableSecuritiesNoncurrent").val.astype(float)  # longTermInvestments
    cash = prepre_sec_dataframe(df,"CashAndCashEquivalentsAtCarryingValue").val.astype(float) 
    total = short_term + long_term + cash
    return total

def total_debt(df):
    return prepre_sec_dataframe(company_facts,"LongTermDebtNoncurrent").val + current_debt(df)

def receivables(df):
    non_trade = prepre_sec_dataframe(df,"NontradeReceivablesCurrent").val.astype(float)
    acc_receivables = prepre_sec_dataframe(df,"AccountsReceivableNetCurrent").val.astype(float)
    return non_trade + acc_receivables
def shareholders_equity(df):
    assets = prepre_sec_dataframe(df,"Assets").val.astype(float)
    liabilities = prepre_sec_dataframe(df,"Liabilities").val.astype(float)
    return assets+liabilities
    
def revenue(df):
    return prepre_sec_dataframe(df,"RevenueFromContractWithCustomerExcludingAssessedTax").val.astype(float)
def current_debt(df):
    short_term_liabilities = prepre_sec_dataframe(df,"CommercialPaper").val.astype(float)
    long_term_debt = prepre_sec_dataframe(df,"LongTermDebtCurrent").val.astype(float)
    return short_term_liabilities + long_term_debt

def free_cash_flow(df):
    return prepre_sec_dataframe(df,"NetCashProvidedByUsedInOperatingActivities").val - prepre_sec_dataframe(df,"PaymentsToAcquirePropertyPlantAndEquipment").val

def inventory(df):
    return prepre_sec_dataframe(df,"InventoryNet").val.astype(float)
def growth(df,col):
    current = df[col]
    previous=  df[previous]
    return ((current / previous) - 1) * 100

def compare(df1,df2):
    return pd.concat([df1,df2.add_suffix("_df2")],axis=1)

In [300]:
company_facts[company_facts.fact==" "]

,end,val,accn,fy,fp,form,frame,fact,start
filed,,,,,,,,,


In [75]:
company_facts[company_facts.frame=="CY2024Q1I"]

,end,val,accn,fy,fp,form,frame,fact,start
filed,,,,,,,,,
2024-03-28,2024-02-29,1.277500e+10,0001618921-24-000035,2024,Q2,10-Q,CY2024Q1I,AccountsPayableCurrent,NaN
2024-03-28,2024-02-29,6.200000e+09,0001618921-24-000035,2024,Q2,10-Q,CY2024Q1I,AccountsReceivableNetCurrent,NaN
2024-03-28,2024-02-29,3.420000e+08,0001618921-24-000035,2024,Q2,10-Q,CY2024Q1I,AccruedIncomeTaxesCurrent,NaN
2024-03-28,2024-02-29,7.522000e+09,0001618921-24-000035,2024,Q2,10-Q,CY2024Q1I,AccruedLiabilitiesCurrent,NaN
2024-03-28,2024-02-29,1.320000e+10,0001618921-24-000035,2024,Q2,10-Q,CY2024Q1I,AccumulatedDepreciationDepletionAndAmortizatio...,NaN
...,...,...,...,...,...,...,...,...,...
2024-03-28,2024-02-29,3.098003e+08,0001618921-24-000035,2024,Q2,10-Q,CY2024Q1I,TreasuryStockCommonShares,NaN
2024-03-28,2024-02-29,2.070500e+10,0001618921-24-000035,2024,Q2,10-Q,CY2024Q1I,TreasuryStockCommonValue,NaN
2024-03-28,2024-02-29,6.090000e+08,0001618921-24-000035,2024,Q2,10-Q,CY2024Q1I,LessorOperatingLeasePaymentsToBeReceived,NaN


In [297]:
df2 = prepre_sec_dataframe(company_facts)
df2.val = df2.val / 10e6
short_term = prepre_sec_dataframe(company_facts[company_facts.fact.str.contains("MarketableSecuritiesCurrent")]).val.astype(float) #ShortTermInvestments
long_term= prepre_sec_dataframe(company_facts[company_facts.fact.str.contains("MarketableSecuritiesNoncurrent")]).val.astype(float)  # longTermInvestments
cash = prepre_sec_dataframe(company_facts[company_facts.fact.str.contains("CashAndCashEquivalentsAtCarryingValue")]).val.astype(float) 
total = short_term + long_term + cash
total


end
2006-09-30             NaN
2007-09-29             NaN
2008-06-28             NaN
2008-09-27             NaN
2008-12-27             NaN
                  ...     
2022-12-31    1.654500e+11
2023-04-01    1.663330e+11
2023-07-01    1.665430e+11
2023-09-30    1.620990e+11
2023-12-30    1.725750e+11
Name: val, Length: 65, dtype: float64

In [298]:
for x in total:
    print(x)

nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
nan
237100000000.0
245035000000.0
225411000000.0
210610000000.0
157077000000.0
167358000000.0
192844000000.0
160263000000.0
153850000000.0
195571000000.0
204373000000.0
193644000000.0
190516000000.0
202596000000.0
192730000000.0
179308000000.0
169109000000.0
165450000000.0
166333000000.0
166543000000.0
162099000000.0
172575000000.0


In [455]:
prepre_sec_dataframe(company_facts,"NetIncomeLoss").val

end
2023-12-30    3.391600e+10
2023-07-01    1.988100e+10
2023-04-01    2.416000e+10
2023-09-30    9.699500e+10
2022-12-31    2.999800e+10
                  ...     
2009-09-26    8.235000e+09
2008-12-27    2.255000e+09
2008-06-28    1.072000e+09
2008-09-27    6.119000e+09
2007-09-29    3.495000e+09
Name: val, Length: 76, dtype: float64

In [456]:
company_facts.loc[company_facts.fact=="NetIncomeLoss"]

,end,val,accn,fy,fp,form,frame,fact,start
filed,,,,,,,,,
2009-10-27,2007-09-29,3.496000e+09,0001193125-09-214859,2009,FY,10-K,NaN,NetIncomeLoss,2006-10-01
2010-01-25,2007-09-29,3.495000e+09,0001193125-10-012091,2009,FY,10-K/A,CY2007,NetIncomeLoss,2006-10-01
2009-07-22,2008-06-28,3.698000e+09,0001193125-09-153165,2009,Q3,10-Q,NaN,NetIncomeLoss,2007-09-30
2009-07-22,2008-06-28,1.072000e+09,0001193125-09-153165,2009,Q3,10-Q,CY2008Q2,NetIncomeLoss,2008-03-30
2009-10-27,2008-09-27,4.834000e+09,0001193125-09-214859,2009,FY,10-K,NaN,NetIncomeLoss,2007-09-30
...,...,...,...,...,...,...,...,...,...
2023-05-05,2023-04-01,2.416000e+10,0000320193-23-000064,2023,Q2,10-Q,CY2023Q1,NetIncomeLoss,2023-01-01
2023-08-04,2023-07-01,7.403900e+10,0000320193-23-000077,2023,Q3,10-Q,NaN,NetIncomeLoss,2022-09-25
2023-08-04,2023-07-01,1.988100e+10,0000320193-23-000077,2023,Q3,10-Q,CY2023Q2,NetIncomeLoss,2023-04-02


,fact,val,accn,fy,fp,form,frame,start
end,,,,,,,,
2006-09-30,CashAndCashEquivalentsAtCarryingValue,6.392000e+09,0001193125-10-012091,2009,FY,10-K/A,CY2006Q3I,None
2006-09-30,StockholdersEquity,9.984000e+09,0001193125-10-012091,2009,FY,10-K/A,CY2006Q3I,None
2007-09-29,AdjustmentsToAdditionalPaidInCapitalSharebased...,2.510000e+08,0001193125-09-214859,2009,FY,10-K,CY2007,2006-10-01
2007-09-29,AdjustmentsToAdditionalPaidInCapitalTaxEffectF...,3.980000e+08,0001193125-09-214859,2009,FY,10-K,CY2007,2006-10-01
2007-09-29,CashAndCashEquivalentsAtCarryingValue,9.352000e+09,0001193125-10-238044,2010,FY,10-K,CY2007Q3I,None
...,...,...,...,...,...,...,...,...
2023-12-30,StockRepurchasedAndRetiredDuringPeriodShares,1.180000e+08,0000320193-24-000006,2024,Q1,10-Q,CY2023Q4,2023-10-01
2023-12-30,StockRepurchasedAndRetiredDuringPeriodValue,2.050000e+10,0000320193-24-000006,2024,Q1,10-Q,CY2023Q4,2023-10-01
2023-12-30,StockholdersEquity,7.410000e+10,0000320193-24-000006,2024,Q1,10-Q,CY2023Q4I,None
